In [4]:
import re, random, math, collections, itertools

In [5]:
def readFiles(sentimentDictionary,sentencesTrain,sentencesTest,sentencesNokia):

    #reading pre-labeled input and splitting into lines
    posSentences = open('rt-polarity.pos', 'r', encoding="ISO-8859-1")
    posSentences = re.split(r'\n', posSentences.read())

    negSentences = open('rt-polarity.neg', 'r', encoding="ISO-8859-1")
    negSentences = re.split(r'\n', negSentences.read())

    posSentencesNokia = open('nokia-pos.txt', 'r')
    posSentencesNokia = re.split(r'\n', posSentencesNokia.read())

    negSentencesNokia = open('nokia-neg.txt', 'r', encoding="ISO-8859-1")
    negSentencesNokia = re.split(r'\n', negSentencesNokia.read())
 
    posDictionary = open('positive-words.txt', 'r', encoding="ISO-8859-1")
    posWordList = re.findall(r"[a-z\-]+", posDictionary.read())

    negDictionary = open('negative-words.txt', 'r', encoding="ISO-8859-1")
    negWordList = re.findall(r"[a-z\-]+", negDictionary.read())

    for i in posWordList:
        sentimentDictionary[i] = 1
    for i in negWordList:
        sentimentDictionary[i] = -1
 
    for i in posSentences:
        if random.randint(1,10)<2:
            sentencesTest[i]="positive"
        else:
            sentencesTrain[i]="positive"

    for i in negSentences:
        if random.randint(1,10)<2:
            sentencesTest[i]="negative"
        else:
            sentencesTrain[i]="negative"

    #create Nokia Datset:
    for i in posSentencesNokia:
            sentencesNokia[i]="positive"
    for i in negSentencesNokia:
            sentencesNokia[i]="negative"

In [6]:
#calculates p(W|Positive), p(W|Negative) and p(W) for all words in training data


def trainBayes(sentencesTrain, pWordPos, pWordNeg, pWord):
    posFeatures = [] # [] initialises a list [array]
    negFeatures = [] 
    freqPositive = {} # {} initialises a dictionary [hash function]
    freqNegative = {}
    dictionary = {}
    posWordsTot = 0
    negWordsTot = 0
    allWordsTot = 0

    #iterate through each sentence/sentiment pair in the training data
    for sentence, sentiment in sentencesTrain.items():
        wordList = re.findall(r"[\w']+", sentence)
        
        for word in wordList: #calculate over unigrams
            allWordsTot += 1 # keeps count of total words in dataset
            if not (word in dictionary):
                dictionary[word] = 1
            if sentiment=="positive" :
                posWordsTot += 1 # keeps count of total words in positive class

                #keep count of each word in positive context
                if not (word in freqPositive):
                    freqPositive[word] = 1
                else:
                    freqPositive[word] += 1    
            else:
                negWordsTot+=1# keeps count of total words in negative class
                
                #keep count of each word in positive context
                if not (word in freqNegative):
                    freqNegative[word] = 1
                else:
                    freqNegative[word] += 1

    for word in dictionary:
        #do some smoothing so that minimum count of a word is 1
        if not (word in freqNegative):
            freqNegative[word] = 1
        if not (word in freqPositive):
            freqPositive[word] = 1

        # Calculate p(word|positive)
        pWordPos[word] = freqPositive[word] / float(posWordsTot)

        # Calculate p(word|negative) 
        pWordNeg[word] = freqNegative[word] / float(negWordsTot)

        # Calculate p(word)
        pWord[word] = (freqPositive[word] + freqNegative[word]) / float(allWordsTot) 



In [7]:
def printAccuracy(dataName,correct,total,correctpos,correctneg,totalpos,totalneg,totalpospred,totalnegpred):
    accuracy = correct/total
    precision_pos = correctpos/totalpospred
    precision_neg = correctneg/totalnegpred
    recall_pos = correctpos/totalpos
    recall_neg = correctneg/totalneg
    F1_score_pos = (2*precision_pos*recall_pos)/(precision_pos+recall_pos)
    F1_score_neg = (2*precision_neg*recall_neg)/(precision_neg+recall_neg)
    
    print(dataName)
    print("Accuracy :",accuracy)
    print("Precision - Positive :",precision_pos)
    print("Precision - Negative :",precision_neg)
    print("Recall - Positive :",recall_pos)
    print("Recall - Negative :",recall_neg)
    print("F1 Score - Positive :",F1_score_pos)
    print("F1 Score - Negative :",F1_score_neg,'\n\n')

In [18]:
PRINT_ERRORS=1

#implement naive bayes algorithm
#INPUTS:
#  sentencesTest is a dictonary with sentences associated with sentiment 
#  dataName is a string (used only for printing output)
#  pWordPos is dictionary storing p(word|positive) for each word
#     i.e., pWordPos["apple"] will return a real value for p("apple"|positive)
#  pWordNeg is dictionary storing p(word|negative) for each word
#  pWord is dictionary storing p(word)
#  pPos is a real number containing the fraction of positive reviews in the dataset
def testBayes(sentencesTest, dataName, pWordPos, pWordNeg, pWord,pPos):
    pNeg=1-pPos

    #These variables will store results
    total=0
    correct=0
    totalpos=0
    totalpospred=0
    totalneg=0
    totalnegpred=0
    correctpos=0
    correctneg=0

    #for each sentence, sentiment pair in the dataset
    for sentence, sentiment in sentencesTest.items():
        wordList = re.findall(r"[\w']+", sentence)#collect all words

        pPosW=pPos
        pNegW=pNeg

        for word in wordList: #calculate over unigrams
            if word in pWord:
                if pWord[word]>0.00000001:
                    pPosW *=pWordPos[word]
                    pNegW *=pWordNeg[word]

        prob=0            
        if pPosW+pNegW >0:
            prob=pPosW/float(pPosW+pNegW)


        total+=1
        if sentiment=="positive":
            totalpos+=1
            if prob>0.5:
                correct+=1
                correctpos+=1
                totalpospred+=1
            else:
                correct+=0
                totalnegpred+=1
                if PRINT_ERRORS:
                    print ("ERROR (pos classed as neg %0.2f):" %prob + sentence)
        else:
            totalneg+=1
            if prob<=0.5:
                correct+=1
                correctneg+=1
                totalnegpred+=1
            else:
                correct+=0
                totalpospred+=1
                if PRINT_ERRORS:
                    print ("ERROR (neg classed as pos %0.2f):" %prob + sentence)

# TODO for Step 2: Add some code here to calculate and print: (1) accuracy; (2) precision and recall for the positive class; 
# (3) precision and recall for the negative class; (4) F1 score;
    printAccuracy(dataName,correct,total,correctpos,correctneg,totalpos,totalneg,totalpospred,totalnegpred)
    

In [19]:

# For each word in the sentence, if the word is in the positive dictionary , it adds 1,
# if it is in the negative dictionary, it subtracts 1.
# If the final score is above a threshold, it classifies as "Positive", otherwise as "Negative"
def testDictionary(sentencesTest, dataName, sentimentDictionary, threshold):
    total=0
    correct=0
    totalpos=0
    totalneg=0
    totalpospred=0
    totalnegpred=0
    correctpos=0
    correctneg=0
    for sentence, sentiment in sentencesTest.items():
        Words = re.findall(r"[\w']+", sentence)
        score=0
        for word in Words:
            if word in sentimentDictionary:
               score+=sentimentDictionary[word]
 
        total+=1
        if sentiment=="positive":
            totalpos+=1
            if score>=threshold:
                correct+=1
                correctpos+=1
                totalpospred+=1
            else:
                correct+=0
                totalnegpred+=1
                if PRINT_ERRORS:
                    print ("ERROR (pos classed as neg %0.2f):" %score + sentence)
        else:
            totalneg+=1
            if score<threshold:
                correct+=1
                correctneg+=1
                totalnegpred+=1
            else:
                correct+=0
                totalpospred+=1
                if PRINT_ERRORS:
                    print ("ERROR (neg classed as pos %0.2f):" %score + sentence)
    
    # TODO for Step 5: Add some code here to calculate and print: (1) accuracy; (2) precision and recall for the positive class; 
    # (3) precision and recall for the negative class; (4) F1 score;
    
    printAccuracy(dataName,correct,total,correctpos,correctneg,totalpos,totalneg,totalpospred,totalnegpred)

In [7]:
intensDimiDict = {
    "very": 2,
    "really": 2,
    "quite": 1.5,
    "somewhat": 0.5,
    "a bit": 0.5,
    "extremely": 3,
    "incredibly": 3,
    "totally": 2.5,
    "absolutely": 2.5,
    "rather": 1.5,
}

negationList = ["not", "no", "never", "none", "neither", "nor", "nobody", "nowhere", "nothing", "hardly", "scarcely", "barely"]

In [38]:
# For each word in the sentence, if the word is in the positive dictionary , it adds 1,
# if it is in the negative dictionary, it subtracts 1.
# If the final score is above a threshold, it classifies as "Positive", otherwise as "Negative"
def testDictionaryImproved_manual(sentencesTest, dataName, sentimentDictionary, threshold, intensDimiDict, negationList):
    total=0
    correct=0
    totalpos=0
    totalneg=0
    totalpospred=0
    totalnegpred=0
    correctpos=0
    correctneg=0
    for sentence, sentiment in sentencesTest.items():
        Words = re.findall(r"[\w']+", sentence)
        score=0
        total = len(sentencesTest.items())
        for i in range(0, len(Words)):
            if Words[i] in sentimentDictionary:
                score += 4 * sentimentDictionary[Words[i]]
                
                ## NEGATION RULES
                if i > 0 and Words[i - 1] in negationList:
                    score *= -1
                    
                if sentiment == 'positive':
                    ## CAPITALIZATION
                    if Words[i].isupper():
                        score += 1
                    
                    ## Intensifier/Diminisher Rule
                    if Words[i] in intensDimiDict:
                        score += intensDimiDict[Words[i]]
                    
                    ## Exclamation Rule
                    if Words[i][-1] == '!':
                        score += 1
                    
                else:
                    ## CAPITALIZATION
                    if Words[i].isupper():
                        score += 1
                    
                    ## Intensifier/Diminisher Rule
                    if Words[i] in intensDimiDict:
                        score -= intensDimiDict[Words[i]]
                    
                    ## Exclamation Rule
                    if Words[i][-1] == '!':
                        score -= 1

        #total+=1
        #print(score)
        if sentiment=="positive":
            totalpos+=1
            if score>=threshold:
                correct+=1
                correctpos+=1
                totalpospred+=1
            else:
                #correct+=0
                totalnegpred+=1
        else:
            totalneg+=1
            if score<threshold:
                correct+=1
                correctneg+=1
                totalnegpred+=1
            else:
                #correct+=0
                totalpospred+=1
    
    # TODO for Step 5: Add some code here to calculate and print: (1) accuracy; (2) precision and recall for the positive class; 
    # (3) precision and recall for the negative class; (4) F1 score;
    
    printAccuracy(dataName,correct,total,correctpos,correctneg,totalpos,totalneg,totalpospred,totalnegpred)

In [28]:
# For each word in the sentence, if the word is in the positive dictionary , it adds 1,
# if it is in the negative dictionary, it subtracts 1.
# If the final score is above a threshold, it classifies as "Positive", otherwise as "Negative"
from afinn import Afinn

def testDictionaryImproved_afinn(sentencesTest, dataName, threshold):
    total=0
    correct=0
    totalpos=0
    totalneg=0
    totalpospred=0
    totalnegpred=0
    correctpos=0
    correctneg=0
    afinn = Afinn()
    for sentence, sentiment in sentencesTest.items():
        Words = re.findall(r"[\w']+", sentence)
        score=0
        total = len(sentencesTest.items())

        for word in Words:
            score += afinn.score(word)

        if sentiment=="positive":
            totalpos+=1
            if score>=threshold:
                correct+=1
                correctpos+=1
                totalpospred+=1
            else:
                #correct+=0
                totalnegpred+=1
        else:
            totalneg+=1
            if score<threshold:
                correct+=1
                correctneg+=1
                totalnegpred+=1
            else:
                #correct+=0
                totalpospred+=1
    
    # TODO for Step 5: Add some code here to calculate and print: (1) accuracy; (2) precision and recall for the positive class; 
    # (3) precision and recall for the negative class; (4) F1 score;
    
    printAccuracy(dataName,correct,total,correctpos,correctneg,totalpos,totalneg,totalpospred,totalnegpred)

In [30]:
# For each word in the sentence, if the word is in the positive dictionary , it adds 1,
# if it is in the negative dictionary, it subtracts 1.
# If the final score is above a threshold, it classifies as "Positive", otherwise as "Negative"
from textblob import TextBlob

def testDictionaryImproved_textblob(sentencesTest, dataName, threshold):
    total=0
    correct=0
    totalpos=0
    totalneg=0
    totalpospred=0
    totalnegpred=0
    correctpos=0
    correctneg=0
    for sentence, sentiment in sentencesTest.items():
        Words = re.findall(r"[\w']+", sentence)
        score=0
        total = len(sentencesTest.items())
        for word in Words:
            word_blob = TextBlob(word)
            score += word_blob.sentiment.polarity
                        

        if sentiment=="positive":
            totalpos+=1
            if score>=threshold:
                correct+=1
                correctpos+=1
                totalpospred+=1
            else:
                #correct+=0
                totalnegpred+=1
        else:
            totalneg+=1
            if score<threshold:
                correct+=1
                correctneg+=1
                totalnegpred+=1
            else:
                #correct+=0
                totalpospred+=1
    
    # TODO for Step 5: Add some code here to calculate and print: (1) accuracy; (2) precision and recall for the positive class; 
    # (3) precision and recall for the negative class; (4) F1 score;
    
    printAccuracy(dataName,correct,total,correctpos,correctneg,totalpos,totalneg,totalpospred,totalnegpred)

In [21]:
lexicon_path = "vader_lexicon.txt"

vader_lexicon = {}

with open(lexicon_path, 'r', encoding='utf-8') as file:
    next(file)
    for line in file:
        fields = line.strip().split('\t')
        if len(fields) == 4:
            word = fields[0]
            sentiment = float(fields[1])
            vader_lexicon[word] = sentiment

In [22]:
# For each word in the sentence, if the word is in the positive dictionary , it adds 1,
# if it is in the negative dictionary, it subtracts 1.
# If the final score is above a threshold, it classifies as "Positive", otherwise as "Negative"
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def testDictionaryImproved_vader(sentencesTest, dataName, threshold,vader_lexicon):
    #total=0
    correct=0
    totalpos=0
    totalneg=0
    totalpospred=0
    totalnegpred=0
    correctpos=0
    correctneg=0
    for sentence, sentiment in sentencesTest.items():
        Words = re.findall(r"[\w']+", sentence)
        score=0
        total = len(sentencesTest.items())
        for word in Words:
            if word in vader_lexicon:
               score+=vader_lexicon[word]
                        

        if sentiment=="positive":
            totalpos+=1
            if score>=threshold:
                correct+=1
                correctpos+=1
                totalpospred+=1
            else:
                #correct+=0
                totalnegpred+=1
        else:
            totalneg+=1
            if score<threshold:
                correct+=1
                correctneg+=1
                totalnegpred+=1
            else:
                #correct+=0
                totalpospred+=1
    
    # TODO for Step 5: Add some code here to calculate and print: (1) accuracy; (2) precision and recall for the positive class; 
    # (3) precision and recall for the negative class; (4) F1 score;
    
    printAccuracy(dataName,correct,total,correctpos,correctneg,totalpos,totalneg,totalpospred,totalnegpred)

In [19]:
# For each word in the sentence, if the word is in the positive dictionary , it adds 1,
# if it is in the negative dictionary, it subtracts 1.
# If the final score is above a threshold, it classifies as "Positive", otherwise as "Negative"
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def testDictionaryImproved_vader_sent(sentencesTest, dataName, threshold):
    #total=0
    correct=0
    totalpos=0
    totalneg=0
    totalpospred=0
    totalnegpred=0
    correctpos=0
    correctneg=0
    sia = SentimentIntensityAnalyzer()
    for sentence, sentiment in sentencesTest.items():
        Words = re.findall(r"[\w']+", sentence)
        score=0
        total = len(sentencesTest.items())
        for word in Words:
            score = sia.polarity_scores(word)['compound']
                        

        if sentiment=="positive":
            totalpos+=1
            if score>=threshold:
                correct+=1
                correctpos+=1
                totalpospred+=1
            else:
                #correct+=0
                totalnegpred+=1
        else:
            totalneg+=1
            if score<threshold:
                correct+=1
                correctneg+=1
                totalnegpred+=1
            else:
                #correct+=0
                totalpospred+=1
    
    # TODO for Step 5: Add some code here to calculate and print: (1) accuracy; (2) precision and recall for the positive class; 
    # (3) precision and recall for the negative class; (4) F1 score;
    
    printAccuracy(dataName,correct,total,correctpos,correctneg,totalpos,totalneg,totalpospred,totalnegpred)

In [39]:
#Print out n most useful predictors
def mostUseful(pWordPos, pWordNeg, pWord, n):
    predictPower={}
    for word in pWord:
        if pWordNeg[word]>0.0000001:
            predictPower[word]=pWordPos[word] / (pWordPos[word] + pWordNeg[word])
            
    sortedPower = sorted(predictPower, key=predictPower.get)
    head, tail = sortedPower[:n], sortedPower[len(predictPower)-n:]
    print ("NEGATIVE:")
    print (head)
    print ("\n\nPOSITIVE:")
    print (tail)
    return [head,tail,sortedPower]

In [9]:
sentimentDictionary={} # {} initialises a dictionary [hash function]
sentencesTrain={}
sentencesTest={}
sentencesNokia={}

#initialise datasets and dictionaries
readFiles(sentimentDictionary,sentencesTrain,sentencesTest,sentencesNokia)

pWordPos={} # p(W|Positive)
pWordNeg={} # p(W|Negative)
pWord={}    # p(W) 


#build conditional probabilities using training data
trainBayes(sentencesTrain, pWordPos, pWordNeg, pWord)

In [17]:
#run naive bayes classifier on datasets
print ("Naive Bayes")
testBayes(sentencesTrain,  "Films (Train Data, Naive Bayes)\t", pWordPos, pWordNeg, pWord,0.5)
testBayes(sentencesTest,  "Films  (Test Data, Naive Bayes)\t", pWordPos, pWordNeg, pWord,0.5)
testBayes(sentencesNokia, "Nokia   (All Data,  Naive Bayes)\t", pWordPos, pWordNeg, pWord,0.7)

Naive Bayes
there is much which has been said in other reviews about the features of this phone , it is a great phone , mine worked without any problems right out of the box . 
i have had the phone for 1 week , the signal quality has been great in the detroit area ( suburbs ) and in my recent road trip between detroit and northern kentucky ( cincinnati ) i experienced perfect signal and reception along i-75 , far superior to at &#38; t 's which does not work along several long stretches on that same route . 
the speaker phone is very functional and i use it in the car , very audible even with freeway noise . 
the infrared is a blessing if you have a previous nokia and want to transfer your old phone book to this phone , saved me hours of re-entering my numbers . 
here 's the brief synopsis : the phone is tiny , cute , feels kind of " plastic-like " ( as if it might break ) , but seems pretty sturdy . 
i do like this phone overall ( would give it a 3.5 ) , but unforutnately , i am retur

In [21]:
#run sentiment dictionary based classifier on datasets
#testDictionary(sentencesTrain,  "Films (Train Data, Rule-Based)\t", sentimentDictionary, -4)
#testDictionary(sentencesTest,  "Films  (Test Data, Rule-Based)\t",  sentimentDictionary, -4)
testDictionary(sentencesNokia, "Nokia   (All Data, Rule-Based)\t",  sentimentDictionary, -3)

ERROR (pos classed as neg -5.00):there is much which has been said in other reviews about the features of this phone , it is a great phone , mine worked without any problems right out of the box . 
ERROR (pos classed as neg -9.00):i have had the phone for 1 week , the signal quality has been great in the detroit area ( suburbs ) and in my recent road trip between detroit and northern kentucky ( cincinnati ) i experienced perfect signal and reception along i-75 , far superior to at &#38; t 's which does not work along several long stretches on that same route . 
ERROR (pos classed as neg -6.00):my favorite features , although there are many , are the speaker phone , the radio and the infrared . 
ERROR (pos classed as neg -5.00):the speaker phone is very functional and i use it in the car , very audible even with freeway noise . 
ERROR (pos classed as neg -6.00):the infrared is a blessing if you have a previous nokia and want to transfer your old phone book to this phone , saved me hours

In [42]:
# print most useful words
[head,tail,powerList]=mostUseful(pWordPos, pWordNeg, pWord, 15)

# find ratio of words in powerlist present in the sentimentDictionary

L = len(powerList)
count = 0
for i in powerList:
    if i in sentimentDictionary:
        count += 1

print("Ratio of words in powerlist present in the sentimentDictionary: ", count/L)

NEGATIVE:
['boring', 'generic', 'mediocre', 'routine', 'unfunny', 'waste', 'badly', 'poorly', 'stale', 'pointless', 'disguise', 'unless', 'mindless', 'shoot', 'tiresome']


POSITIVE:
['intense', 'detailed', 'wonderfully', 'realistic', 'chilling', 'refreshingly', 'gem', 'riveting', 'mesmerizing', 'haunting', 'unique', 'refreshing', 'inventive', 'wonderful', 'engrossing']
Ratio of words in powerlist present in the sentimentDictionary:  0.16837560712358338


In [29]:
#run sentiment dictionary based classifier on datasets
testDictionaryImproved_afinn(sentencesTrain,  "Films (Train Data, Rule-Based)\t", -5)
testDictionaryImproved_afinn(sentencesTest,  "Films  (Test Data, Rule-Based)\t", -10)
testDictionaryImproved_afinn(sentencesNokia, "Nokia   (All Data, Rule-Based)\t", 5)

Films (Train Data, Rule-Based)	
Accuracy : 0.5164594001463058
Precision - Positive : 0.5092701746865288
Precision - Negative : 0.7983193277310925
Recall - Positive : 0.99
Recall - Negative : 0.0398406374501992
F1 Score - Positive : 0.6725638666761022
F1 Score - Negative : 0.07589374875174755 


Films  (Test Data, Rule-Based)	
Accuracy : 0.4858447488584475
Precision - Positive : 0.48581884720951507
Precision - Negative : 0.5
Recall - Positive : 0.9981203007518797
Recall - Negative : 0.0017761989342806395
F1 Score - Positive : 0.6535384615384615
F1 Score - Negative : 0.003539823008849558 


Nokia   (All Data, Rule-Based)	
Accuracy : 0.4398496240601504
Precision - Positive : 0.9743589743589743
Precision - Negative : 0.34801762114537443
Recall - Positive : 0.20430107526881722
Recall - Negative : 0.9875
F1 Score - Positive : 0.33777777777777784
F1 Score - Negative : 0.5146579804560261 




In [33]:
#run sentiment dictionary based classifier on datasets
testDictionaryImproved_textblob(sentencesTrain,  "Films (Train Data, Rule-Based)\t", 0)
testDictionaryImproved_textblob(sentencesTest,  "Films  (Test Data, Rule-Based)\t", 0)
testDictionaryImproved_textblob(sentencesNokia, "Nokia   (All Data, Rule-Based)\t", 0)

Films (Train Data, Rule-Based)	
Accuracy : 0.6145887762566621
Precision - Positive : 0.5787535410764872
Precision - Negative : 0.7154244719011559
Recall - Positive : 0.85125
Recall - Negative : 0.37638918012161876
F1 Score - Positive : 0.6890387858347385
F1 Score - Negative : 0.4932673811486672 


Films  (Test Data, Rule-Based)	
Accuracy : 0.6100456621004566
Precision - Positive : 0.5653798256537983
Precision - Negative : 0.7328767123287672
Recall - Positive : 0.8533834586466166
Recall - Negative : 0.38010657193605685
F1 Score - Positive : 0.6801498127340824
F1 Score - Negative : 0.5005847953216375 


Nokia   (All Data, Rule-Based)	
Accuracy : 0.7706766917293233
Precision - Positive : 0.7777777777777778
Precision - Negative : 0.7317073170731707
Recall - Positive : 0.9408602150537635
Recall - Negative : 0.375
F1 Score - Positive : 0.8515815085158152
F1 Score - Negative : 0.4958677685950414 




In [23]:
#run sentiment dictionary based classifier on datasets
testDictionaryImproved_vader(sentencesTrain,  "Films (Train Data, Rule-Based)\t", 0,vader_lexicon)
testDictionaryImproved_vader(sentencesTest,  "Films  (Test Data, Rule-Based)\t", 0,vader_lexicon)
testDictionaryImproved_vader(sentencesNokia, "Nokia   (All Data, Rule-Based)\t", 0,vader_lexicon)

Films (Train Data, Rule-Based)	
Accuracy : 0.5994406463642014
Precision - Positive : 0.5682630450321658
Precision - Negative : 0.6814591951861602
Recall - Positive : 0.8243467440895894
Recall - Negative : 0.375
F1 Score - Positive : 0.6727595836506728
F1 Score - Negative : 0.4837805366439728 


Films  (Test Data, Rule-Based)	
Accuracy : 0.6075322101090188
Precision - Positive : 0.5762483130904184
Precision - Negative : 0.6940298507462687
Recall - Positive : 0.8388998035363457
Recall - Negative : 0.372
F1 Score - Positive : 0.6832
F1 Score - Negative : 0.484375 


Nokia   (All Data, Rule-Based)	
Accuracy : 0.7969924812030075
Precision - Positive : 0.7946428571428571
Precision - Negative : 0.8095238095238095
Recall - Positive : 0.956989247311828
Recall - Negative : 0.425
F1 Score - Positive : 0.8682926829268293
F1 Score - Negative : 0.5573770491803278 




In [39]:
#run sentiment dictionary based classifier on datasets
testDictionaryImproved_manual(sentencesTrain,  "Films (Train Data, Rule-Based)\t",sentimentDictionary, 0,intensDimiDict, negationList)
testDictionaryImproved_manual(sentencesTest,  "Films  (Test Data, Rule-Based)\t",sentimentDictionary, 0,intensDimiDict, negationList)
testDictionaryImproved_manual(sentencesNokia, "Nokia   (All Data, Rule-Based)\t",sentimentDictionary, 0,intensDimiDict, negationList)

Films (Train Data, Rule-Based)	
Accuracy : 0.5303584491587418
Precision - Positive : 0.6303236797274276
Precision - Negative : 0.5163787969029184
Recall - Positive : 0.15416666666666667
Recall - Negative : 0.9089955965611239
F1 Score - Positive : 0.24774020756611986
F1 Score - Negative : 0.6586144029170465 


Films  (Test Data, Rule-Based)	
Accuracy : 0.5324200913242009
Precision - Positive : 0.578125
Precision - Negative : 0.5263702171664943
Recall - Positive : 0.13909774436090225
Recall - Negative : 0.9040852575488455
F1 Score - Positive : 0.22424242424242424
F1 Score - Negative : 0.6653594771241831 


Nokia   (All Data, Rule-Based)	
Accuracy : 0.48120300751879697
Precision - Positive : 0.8243243243243243
Precision - Negative : 0.3489583333333333
Recall - Positive : 0.3279569892473118
Recall - Negative : 0.8375
F1 Score - Positive : 0.46923076923076923
F1 Score - Negative : 0.4926470588235294 


